# Классификация ирисов (ml - DataFrame api)
+ Импорт данных из hdfs
+ Разделение выборки на обучающую и тестовую
+ Обучение модели многоклассовой классификации с помощью логистической регрессии
+ Оценка модели на тестовой выборке
+ Random Forest

## Импорт данных из hdfs

In [1]:
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.linalg.Vectors

val data = sc.textFile("/user/supp.bda08/iris.csv")
    .filter(_ != "0,1,2,3,0")
    .map(r => {
        val row = r.split(",")
        val vector = row.slice(0,4).map(_.toDouble)
        val label  = row(4).toInt
        LabeledPoint(label, Vectors.dense(vector))
    })
    .filter(point => {point.label != 2.0})
    .toDF("label", "features")
    
println(data.count)
data.show(5)

Waiting for a Spark session to start...

100


data: org.apache.spark.sql.DataFrame = [label: double, features: vector]


+-----+-----------------+
|label|         features|
+-----+-----------------+
|  0.0|[5.1,3.5,1.4,0.2]|
|  0.0|[4.9,3.0,1.4,0.2]|
|  0.0|[4.7,3.2,1.3,0.2]|
|  0.0|[4.6,3.1,1.5,0.2]|
|  0.0|[5.0,3.6,1.4,0.2]|
+-----+-----------------+
only showing top 5 rows



# Разделение выборки на обучающую и тестовую 

In [2]:
val splited = data.randomSplit(weights = Array(0.85, 0.15), seed = 11)

val train = splited(0).cache()
val test = splited(1)

splited = Array([label: double, features: vector], [label: double, features: vector])
train = [label: double, features: vector]
test = [label: double, features: vector]


[label: double, features: vector]

# Обучение модели многоклассовой классификации
Создадим модель Логистической регресии с параметрами:
+ Кол-во итераций - 100
+ Параметр регуляризации - 0.3
+ ElasticNet параметр - 0.2 

$$\text{ElasticNet: }\;\; \alpha \left(\lambda \lVert \mathbf w \rVert_1 \right) + (1 - \alpha)\left(\frac{\lambda}{2}\lVert \mathbf w \rVert_2^2\right), \alpha \in [0,1]$$

Для того чтобы получить модель классификации с более чем 2 классами, нужно создать модель OneVsRest, задать ей классификатор и обучаться с её помощью.

In [3]:
import org.apache.spark.ml.classification.{OneVsRest, LogisticRegression}
import org.apache.spark.ml.util.MetadataUtils

val classifier = new LogisticRegression() 
      .setMaxIter(100)
      .setRegParam(0.3)
      .setElasticNetParam(0.2)

val ovr = new OneVsRest()
ovr.setClassifier(classifier)

val model = ovr.fit(train)

val preds = model.transform(test)

preds.show(5)

classifier: org.apache.spark.ml.classification.LogisticRegression = logreg_1fa894baf5a1
ovr: org.apache.spark.ml.classification.OneVsRest = oneVsRest_8cd48ca2465b
model: org.apache.spark.ml.classification.OneVsRestModel = oneVsRest_8cd48ca2465b
preds: org.apache.spark.sql.DataFrame = [label: double, features: vector, prediction: double]


+-----+-----------------+----------+
|label|         features|prediction|
+-----+-----------------+----------+
|  0.0|[4.7,3.2,1.3,0.2]|       0.0|
|  0.0|[4.8,3.0,1.4,0.1]|       0.0|
|  0.0|[5.0,3.2,1.2,0.2]|       0.0|
|  0.0|[5.0,3.4,1.5,0.2]|       0.0|
|  0.0|[5.3,3.7,1.5,0.2]|       0.0|
+-----+-----------------+----------+
only showing top 5 rows



# Оценка модели на тестовой выборке

In [4]:
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

val evaluator = new MulticlassClassificationEvaluator()

val f1_score = evaluator.evaluate(preds)

println(s"f1 score = ${f1_score}")

evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_99c9f72449a2
f1_score: Double = 1.0


f1 score = 1.0


# Random Forest
Аналогично можно работать с другими моделями. Пример - Random Forest

In [5]:
import org.apache.spark.ml.classification.RandomForestClassifier

val classifier = new RandomForestClassifier()
    .setNumTrees(10)

ovr.setClassifier(classifier)

val model = ovr.fit(train)

val preds = model.transform(test)

val f1_score = evaluator.evaluate(preds)

println(s"f1 score = ${f1_score}")

classifier: org.apache.spark.ml.classification.RandomForestClassifier = rfc_54335529c05d
model: org.apache.spark.ml.classification.OneVsRestModel = oneVsRest_8cd48ca2465b
preds: org.apache.spark.sql.DataFrame = [label: double, features: vector, prediction: double]
f1_score: Double = 1.0


f1 score = 1.0
